In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/english63/train.csv
/kaggle/input/cbenglish/cyber_bully_english.csv
/kaggle/input/cb-english1/cyber_bully_english.csv
/kaggle/input/kannada34/Final Kannada (2).csv
/kaggle/input/english45/train.csv
/kaggle/input/hinglish64/Hinglish final.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
%config Completer.use_jedi = False # this to force autocompletion 

In [3]:
df = pd.read_csv('../input/kannada34/Final Kannada (2).csv')
df.head()

,Classifications,Comments
0,0,Namma galli goo bandiddru ashwatthama
1,0,@vinay vinu ಅಶ್ವತ್ಥಾಮ ಹೇಗೆ ಚಿರಂಜೀವಿಯೋ ಹಾಗೆಯೇ ಭ...
2,0,@vinay vinu ಇವ್ನು ಯಾರ್ ಗುರು ನಂದೆಲ್ಲಿ ಇಡ್ಲಿ ಅಂತ...
3,0,Ayyo guru... Yelli tanka jana moodnambke bidal...
4,0,@vinay vinu Hindu Muslim Cristian etc etc... ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7729 entries, 0 to 7728
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Classifications  7729 non-null   int64 
 1   Comments         7729 non-null   object
dtypes: int64(1), object(1)
memory usage: 120.9+ KB


In [5]:
df.shape

(7729, 2)

In [6]:
pip install text-hammer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.1
    Uninstalling beautifulsoup4-4.11.1:
      Successfully uninstalled beautifulsoup4-4.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import text_hammer as th

In [8]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
    df[column] = df[column].progress_apply(lambda x: th.cont_exp(x)) #you're -> you are; i'm -> i am
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_stopwords(x))
#     df[column] = df[column].progress_apply(lambda x:th.spelling_correction(x))
  
    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
    return(df)

In [9]:
cleaned_df = text_preprocessing(df, "Comments")

  0%|          | 0/7729 [00:00<?, ?it/s]

  0%|          | 0/7729 [00:00<?, ?it/s]

  0%|          | 0/7729 [00:00<?, ?it/s]

  0%|          | 0/7729 [00:00<?, ?it/s]

  0%|          | 0/7729 [00:00<?, ?it/s]

  0%|          | 0/7729 [00:00<?, ?it/s]

  0%|          | 0/7729 [00:00<?, ?it/s]

  0%|          | 0/7729 [00:00<?, ?it/s]

In [10]:
cleaned_df.Comments

0                   namma galli goo bandiddru ashwatthama
1                           vinay vinu                   
2                                        vinay vinu      
3       ayyo guru yelli tanka jana moodnambke bidalvo ...
4       vinay vinu hindu muslim cristian on yella naav...
                              ...                        
7724                          pakka mas nam kannada movie
7725                                                     
7726                                                     
7727                       idu cheritre shrustiso avatara
7728                               yen song gurubenki asn
Name: Comments, Length: 7729, dtype: object

In [11]:
import nltk



### Calculating the word frequency by using nltk
words_list = []
for sentence in cleaned_df.Comments:
    words_list.extend(nltk.word_tokenize(sentence))
freq_dist = nltk.FreqDist(words_list)
freq_dist.most_common(20)

[('super', 1145),
 ('song', 657),
 ('bro', 538),
 ('sir', 525),
 ('movie', 471),
 ('guru', 418),
 ('like', 346),
 ('n', 346),
 ('kannada', 252),
 ('boss', 241),
 ('fan', 233),
 ('madi', 220),
 ('ge', 211),
 ('good', 183),
 ('video', 174),
 ('anna', 170),
 ('d', 169),
 ('film', 169),
 ('love', 154),
 ('e', 148)]

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(cleaned_df.Comments, cleaned_df.Classifications, test_size = 0.2, stratify = cleaned_df.Classifications
                                                 ,random_state = 42)

In [13]:
from keras.preprocessing.text import Tokenizer
num_words = 10000 # this means 15000 unique words can be taken 
tokenizer=Tokenizer(num_words,lower=True)
df_total = pd.concat([X_train, X_test], axis = 0)
tokenizer.fit_on_texts(df_total)

In [14]:
len(tokenizer.word_index)

11400

In [15]:
from wordcloud import WordCloud
import wordcloud
df['word_length'] = df['Comments'].apply(lambda x:len(x.split()))
cleaned_df.word_length.max()

83

In [16]:
from keras.preprocessing.sequence import pad_sequences

X_train_ =tokenizer.texts_to_sequences(X_train)
X_train_pad=pad_sequences(X_train_,maxlen=449,padding='post')
X_test_ = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_, maxlen = 449, padding = 'post')

In [17]:
print(X_train_pad.shape,X_test_pad.shape)

(6183, 449) (1546, 449)


In [18]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM,CuDNNGRU
from tensorflow.keras.layers import Dropout

In [19]:
from keras.models import Sequential
EMBEDDING_DIM = 100 # this means the embedding layer will create  a vector in 100 dimension
model = Sequential()
model.add(Embedding(input_dim = num_words,# the whole vocabulary size 
                          output_dim = EMBEDDING_DIM, # vector space dimension
                          input_length= X_train_pad.shape[1] # max_len of text sequence
                          ))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics = 'accuracy')

2022-07-24 11:22:01.434013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 11:22:01.438298: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 11:22:01.438996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-24 11:22:01.440572: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [20]:
from keras.callbacks import EarlyStopping, ModelCheckpoint


es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 10)
mc = ModelCheckpoint('./model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

In [21]:
history_embedding = model.fit(X_train_pad,y_train, epochs = 35, batch_size = 120, validation_data=(X_test_pad, y_test),verbose = 1, callbacks= [es, mc]  )

2022-07-24 11:22:04.639501: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/35


2022-07-24 11:22:09.248791: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


52/52 [==============================] - 24s 327ms/step - loss: 0.4956 - accuracy: 0.7925 - val_loss: 0.4168 - val_accuracy: 0.8118

Epoch 00001: val_accuracy improved from -inf to 0.81177, saving model to ./model.h5
Epoch 2/35
52/52 [==============================] - 15s 286ms/step - loss: 0.3169 - accuracy: 0.8789 - val_loss: 0.3633 - val_accuracy: 0.8603

Epoch 00002: val_accuracy improved from 0.81177 to 0.86028, saving model to ./model.h5
Epoch 3/35
52/52 [==============================] - 15s 287ms/step - loss: 0.2065 - accuracy: 0.9303 - val_loss: 0.4215 - val_accuracy: 0.8596

Epoch 00003: val_accuracy did not improve from 0.86028
Epoch 4/35
52/52 [==============================] - 15s 286ms/step - loss: 0.1442 - accuracy: 0.9547 - val_loss: 0.4681 - val_accuracy: 0.8538

Epoch 00004: val_accuracy did not improve from 0.86028
Epoch 5/35
52/52 [==============================] - 15s 284ms/step - loss: 0.1132 - accuracy: 0.9646 - val_loss: 0.5111 - val_accuracy: 0.8441

Epoch 0000

In [22]:
import gensim.downloader as api
glove_gensim  = api.load('glove-wiki-gigaword-100')

[===============================================---] 95.5% 122.3/128.1MB downloaded


In [23]:
glove_gensim['cat'].shape[0] 

100

In [24]:
vector_size = 100
gensim_weight_matrix = np.zeros((num_words ,vector_size))
gensim_weight_matrix.shape

for word, index in tokenizer.word_index.items():
    if index < num_words: # since index starts with zero 
        if word in glove_gensim.key_to_index:
            gensim_weight_matrix[index] = glove_gensim[word]
        else:
            gensim_weight_matrix[index] = np.zeros(100)

In [25]:
gensim_weight_matrix.shape

(10000, 100)

In [26]:
EMBEDDING_DIM = 100 # this means the embedding layer will create  a vector in 100 dimension
model_gensim = Sequential()
model_gensim.add(Embedding(input_dim = num_words,# the whole vocabulary size 
                          output_dim = EMBEDDING_DIM, # vector space dimension
                          input_length= X_train_pad.shape[1], # max_len of text sequence
                          weights = [gensim_weight_matrix],trainable = False))
model_gensim.add(Dropout(0.2))
model_gensim.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
model_gensim.add(Dropout(0.2))
model_gensim.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
model_gensim.add(Dropout(0.2))
model_gensim.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
model_gensim.add(Dense(1, activation = 'sigmoid'))
model_gensim.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics = 'accuracy')

In [27]:
model_gensim.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 449, 100)          1000000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 449, 100)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 449, 200)          161600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 449, 200)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 449, 400)          643200    
_________________________________________________________________
dropout_5 (Dropout)          (None, 449, 400)          0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200)              

In [28]:
from keras.callbacks import EarlyStopping, ModelCheckpoint


es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
mc = ModelCheckpoint('./model_gensim.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

In [29]:
history_gensim = model_gensim.fit(X_train_pad,y_train, epochs = 35, batch_size = 120, validation_data=(X_test_pad, y_test),verbose = 1, callbacks= [es, mc]  )

Epoch 1/35
52/52 [==============================] - 19s 294ms/step - loss: 0.4760 - accuracy: 0.7998 - val_loss: 0.4146 - val_accuracy: 0.8312

Epoch 00001: val_accuracy improved from -inf to 0.83118, saving model to ./model_gensim.h5
Epoch 2/35
52/52 [==============================] - 15s 281ms/step - loss: 0.4139 - accuracy: 0.8282 - val_loss: 0.4167 - val_accuracy: 0.8376

Epoch 00002: val_accuracy improved from 0.83118 to 0.83765, saving model to ./model_gensim.h5
Epoch 3/35
52/52 [==============================] - 14s 277ms/step - loss: 0.3987 - accuracy: 0.8378 - val_loss: 0.4042 - val_accuracy: 0.8357

Epoch 00003: val_accuracy did not improve from 0.83765
Epoch 4/35
52/52 [==============================] - 14s 278ms/step - loss: 0.3807 - accuracy: 0.8460 - val_loss: 0.4117 - val_accuracy: 0.8435

Epoch 00004: val_accuracy improved from 0.83765 to 0.84347, saving model to ./model_gensim.h5
Epoch 5/35
52/52 [==============================] - 14s 277ms/step - loss: 0.3683 - accura

In [30]:
model_gensim.evaluate(X_test_pad, y_test) 

49/49 [==============================] - 4s 70ms/step - loss: 0.4412 - accuracy: 0.8486


[0.44122377038002014, 0.8486416339874268]

In [31]:
model.evaluate(X_test_pad, y_test)

49/49 [==============================] - 3s 66ms/step - loss: 0.7019 - accuracy: 0.8454


[0.7019348740577698, 0.8454074859619141]

In [32]:
y_pred = np.where(model.predict(X_test_pad)>.5,1,0)

In [33]:
from sklearn import metrics
print(metrics.classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.48      0.66      0.55       225
           1       0.94      0.88      0.91      1321

    accuracy                           0.85      1546
   macro avg       0.71      0.77      0.73      1546
weighted avg       0.87      0.85      0.86      1546



In [34]:
def predict(sentence):
    sentence_lst=[]
    sentence_lst.append(sentence)
    sentence_seq=tokenizer.texts_to_sequences(sentence_lst)
    sentence_padded=pad_sequences(sentence_seq,maxlen=42,padding='post')
    probability = (model.predict(sentence_padded))[0][0]
    if probability > 0.5:
        print(f'Normal| {np.round(probability*100,2)}%')
    else:
        print(f'Cyberbully| {100 - (np.round(probability*100,2))}%')

In [35]:
predict(str(input('Enter the Sentence:')))


Enter the Sentence: ey hodthbidthini


Normal| 86.62%


In [36]:
predict(str(input('Enter the Sentence:')))

Enter the Sentence: hengidiya?


Normal| 86.62%


In [37]:
predict(str(input('Enter the Sentence:')))

Enter the Sentence: baaro hogona 


Normal| 99.16%


In [38]:
predict(str(input('Enter the Sentence:')))

Enter the Sentence: bewarsi nin hesaru eniu?


Cyberbully| 99.4%


In [39]:
predict(str(input('Enter the Sentence:')))

Enter the Sentence: gante yeshtu ayithu?


Normal| 96.07%


In [40]:
predict(str(input('Enter the Sentence:')))

Enter the Sentence: loafer yakke badididya?


Cyberbully| 66.94%


In [41]:
predict(str(input('Enter the Sentence:')))

KeyboardInterrupt: Interrupted by user

In [ ]:
predict(str(input('Enter the Sentence:')))

In [ ]:
predict(str(input('Enter the Sentence:')))


In [ ]:
predict(str(input('Enter the Sentence:')))

In [ ]:
predict(str(input('Enter the Sentence:')))

In [ ]:
predict(str(input('Enter the Sentence:')))

In [ ]:
predict(str(input('Enter the Sentence:')))


In [ ]:
predict(str(input('Enter the Sentence:')))